In [1]:
import numpy as np
from itertools import count
from collections import deque
from queue import PriorityQueue

In [2]:
class Queue:
    def __init__(self):
        self.queue = deque([])

    def isEmpty(self):
        return len(self.queue) == 0

    def push(self, item):
        self.queue.append(item)

    def pop(self):
        return self.queue.popleft()

    def peek(self):
        return self.queue[0]

    def size(self):
        return len(self.queue)

In [3]:
class Board:
    # data: numpy array containing board data
    # parent: a reference to the parent node
    def __init__(self, data=None, parent=None):
        self.data = data
        self.parent = parent
    
    # build a random board of shape (n0, n1)
    def build(self, n0, n1):
        data = np.arange(n0*n1)
        np.random.shuffle(data)
        self.data = data.reshape(n0,n1)
        return self
    
    # return depth of current board
    def depth(self):
        if self.parent is not None:
            return self.parent.depth() + 1
        else:
            return 0


class TileGame:
    # Get the children of the current board
    def get_available_moves(board):
        available_moves = []
        for i in range(board.data.size):
            coords = np.where(board.data==i)
            if coords[0] > 0:
                available_moves.append(((coords[0],coords[1]),(coords[0]-1,coords[1])))
            if coords[0] < board.data.shape[0]-1:
                available_moves.append(((coords[0],coords[1]),(coords[0]+1,coords[1])))
            if coords[1] > 0:
                available_moves.append(((coords[0],coords[1]),(coords[0],coords[1]-1)))
            if coords[1] < board.data.shape[1]-1:
                available_moves.append(((coords[0],coords[1]),(coords[0],coords[1]+1)))
        return available_moves
  
    # Return a new board with the specified move applied
    def examine_move(board, move):
        data = board.data.copy()
        data[move[0][0],move[0][1]], data[move[1][0],move[1][1]] = data[move[1][0],move[1][1]], data[move[0][0],move[0][1]]
        return Board(data=data, parent=board)
    
    # Check if the specified board is in the goal state
    def is_goal_state(board):
        goal = np.reshape(np.arange(board.data.size),board.data.shape)
        return np.all(board.data==goal)
    
    # Returns the 'path' from the starting board state to goal board state
    def extract_solution(board):    
        solution = []
        solution.append(board)
        while board.parent is not None:
            board = board.parent
            solution.append(board)
        return solution[::-1]
    
    # Prints the 'path' returned by 'extract_solution(board)'
    def print_solution(solution):
        for board in solution:
            print(board.data)
            print()

In [11]:
def heuristic_misplaced_tiles(board):
    goal_board = Board(data=np.reshape(np.arange(board.data.size),board.data.shape))
    heuristic_value = np.sum(board.data!=goal_board.data)
    return heuristic_value/2

# TODO: Heuristic (
def heuristic(board):
    ### TODO: START OF YOUR CODE ###
    ### NOTE: Your heuristic must be admissible AND consistent 
    goal_board = Board(data=np.reshape(np.arange(board.data.size),board.data.shape))
    heuristic_Euclidean_value = np.sqrt(np.sum((board.data- goal_board.data)**2))
    return heuristic_Euclidean_value
    ### TODO: END OF YOUR CODE ###

# TODO: A*
# Hint: You'll need to use a priority queue (queue.PriorityQueue or heapq).
#       Example:
#                frontier = PriorityQueue()
#                ...
#                frontier.put((next_board.depth()+heuristic(next_board), next(unique), next_board))
#       The queue is prioritized by the first value of tuple 'next_board.depth()+heuristic(next_board)'
#       The second value of the tuple 'next(unique)' will simply break ties between queue items where
#       the first value of the tuple is identical.
unique = count()
def astar(board, heuristic):
    ### TODO: START OF YOUR CODE ###
    ### NOTE: Your solution must use an extended set. That is, you should maintain
    ###       a list/set of board states that have already been extended. Board states 
    ###       that have already been extended should not be extended again. Practically, 
    ###       a board state is said to have been extended once its children have 
    ###       been examined. Return the size of the extended set as the second
    ###       return parameter. The first return parameter should be your solution
    ###       (see the HINT below).
    ### HINT: To add a board state to the extended set, add board.data.tostring()
    ###       instead of adding the board object itself. (Why?)
    ### HINT: Use TileGame.extract_solution(board) to generate the corresponding 
    ###       solution from the final board state

    # a node is said to have been 'expanded' when its
    # children have been added to the frontier
    frontier = PriorityQueue()
    frontier.put((board.depth(),next(unique),board))

    expanded_set = set()

    while not frontier.empty():
        current_board = frontier.get()[2]
        if TileGame.is_goal_state(current_board):
            return TileGame.extract_solution(current_board), len(expanded_set)
        elif current_board.data.tostring() not in expanded_set:
            for move in TileGame.get_available_moves(current_board):
                next_board = TileGame.examine_move(current_board,move)
                if next_board.data.tostring() not in expanded_set:
                    frontier.put((next_board.depth()+heuristic(next_board),next(unique),next_board))
            expanded_set.add(current_board.data.tostring())
    raise Exception("Unreachable")

# NOTE: For the specified 2x3 board, your solution should expand no more than 100
#       nodes. The better your heuristic, the fewer nodes you'll need to extend.
#       Our solution extends 11 nodes. Can you do better?

print('2x3 - A*:')
board = Board(data=np.array([[5,0,3],[4,1,2]]))
solution, nodes_expanded = astar(board, heuristic)
print('Nodes Expanded:', nodes_expanded)
print('Solution:')
TileGame.print_solution(solution)

print('3x3 - A*:')
board = Board().build(3,3)
solution, nodes_expanded = astar(board, heuristic)
print('Nodes Expanded:', nodes_expanded)
print('Solution:')
TileGame.print_solution(solution)





2x3 - A*:
Nodes Expanded: 5
Solution:
[[5 0 3]
 [4 1 2]]

[[0 5 3]
 [4 1 2]]

[[0 1 3]
 [4 5 2]]

[[0 1 2]
 [4 5 3]]

[[0 1 2]
 [4 3 5]]

[[0 1 2]
 [3 4 5]]

3x3 - A*:
Nodes Expanded: 16
Solution:
[[5 7 4]
 [6 8 3]
 [0 1 2]]

[[5 7 4]
 [6 1 3]
 [0 8 2]]

[[5 7 4]
 [0 1 3]
 [6 8 2]]

[[5 1 4]
 [0 7 3]
 [6 8 2]]

[[0 1 4]
 [5 7 3]
 [6 8 2]]

[[0 1 4]
 [5 7 3]
 [6 2 8]]

[[0 1 4]
 [5 2 3]
 [6 7 8]]

[[0 1 4]
 [5 3 2]
 [6 7 8]]

[[0 1 2]
 [5 3 4]
 [6 7 8]]

[[0 1 2]
 [3 5 4]
 [6 7 8]]

[[0 1 2]
 [3 4 5]
 [6 7 8]]



In [5]:
# An example of an uninformed solution
def bfs(board):
    frontier = Queue()
    frontier.push(board)
    
    # a node is said to have been 'expanded' when its
    # children have been added to the frontier
    expanded_set = set()
    
    while not frontier.isEmpty():
        current_board = frontier.pop()
        if TileGame.is_goal_state(current_board):
            return TileGame.extract_solution(current_board), len(expanded_set)
        elif current_board.data.tostring() not in expanded_set:
            for move in TileGame.get_available_moves(current_board):
                next_board = TileGame.examine_move(current_board,move)
                if next_board.data.tostring() not in expanded_set:
                    frontier.push(next_board)
            expanded_set.add(current_board.data.tostring())
    raise Exception("Unreachable")


board = Board(data=np.array([[4, 2, 3], [0, 5, 1]]))

solution, nodes_expanded = bfs(board)
print('Nodes Expanded:', nodes_expanded)
print('Solution:')
TileGame.print_solution(solution)

Nodes Expanded: 355
Solution:
[[4 2 3]
 [0 5 1]]

[[0 2 3]
 [4 5 1]]

[[0 2 1]
 [4 5 3]]

[[0 1 2]
 [4 5 3]]

[[0 1 2]
 [4 3 5]]

[[0 1 2]
 [3 4 5]]

